In [19]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso
)
%matplotlib inline

In [20]:
train = pd.read_csv('./input/train.csv') #訓練データ
test = pd.read_csv('./input/test.csv') #テストデータ
# train = train.drop("Id", axis = 1)
target = np.log1p(train["SalePrice"])
train = train.drop("SalePrice", axis = 1)


In [21]:
train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,1,2008,WD,Normal


In [22]:
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [23]:
numerics = train.select_dtypes(include='number').columns.tolist()
numerics



['Id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [24]:
# カテゴリーのカラムを配列に
origin_categoricals = train.select_dtypes(exclude='number').columns.tolist()

# 配列のカラムを丸ごとダミー化
import category_encoders
ce = category_encoders.OrdinalEncoder(cols=origin_categoricals,handle_unknown='impute')
train[origin_categoricals] = ce.fit_transform(train[origin_categoricals])


# train["SalePrice"]

In [25]:
low_fis = ['SaleCondition','LowQualFinSF', 'SaleType', 'Heating', 'GarageQual', 'BsmtFinType2', 'GarageCond',
     'RoofMatl', '3SsnPorch', 'PoolArea', 'Condition2', 'Functional', 'PoolQC', 'Utilities', 'Street', 'MiscVal',
           'BldgType', 'MiscFeature', 'RoofStyle', 'Alley',
            'MSZoning', 'BsmtHalfBath', 'Foundation', 'Fence', 'KitchenAbvGr', 'HouseStyle', 'GarageType', 
            'MasVnrType', 'Condition1', 'BsmtQual', 'PavedDrive', 'LotConfig', 'LandSlope', 'ExterCond', 
            'Electrical', 'BsmtFinSF2', 'BsmtCond', 'BsmtExposure', 'CentralAir', 'Exterior2nd'
          ]

features = origin_categoricals + numerics
categoricals = origin_categoricals.copy()

for low_fi in low_fis:
    features.remove(low_fi)
    if low_fi in categoricals:
        categoricals.remove(low_fi)
    
train = train[features]

In [26]:
train

,LotShape,LandContour,Neighborhood,Exterior1st,ExterQual,BsmtFinType1,HeatingQC,KitchenQual,FireplaceQu,GarageFinish,...,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,MoSold,YrSold
0,1,1,1,1,1,1,1,1,1,1,...,0,2003.0,2,548,0,61,0,0,2,2008
1,1,1,2,2,2,2,1,2,2,1,...,1,1976.0,2,460,298,0,0,0,5,2007
2,2,1,1,1,1,1,1,1,2,1,...,1,2001.0,2,608,0,42,0,0,9,2008
3,2,1,3,3,2,2,2,1,3,2,...,1,1998.0,3,642,0,35,272,0,2,2006
4,2,1,4,1,1,1,1,1,2,1,...,1,2000.0,3,836,192,84,0,0,12,2008
5,2,1,5,1,2,1,1,2,1,2,...,0,1993.0,2,480,40,30,0,0,10,2009
6,1,1,6,1,1,1,1,1,3,1,...,1,2004.0,2,636,255,57,0,0,8,2007
7,2,1,7,4,2,2,1,2,2,1,...,2,1973.0,2,484,235,204,228,0,11,2009
8,1,1,8,5,2,3,2,2,2,2,...,2,1931.0,2,468,90,0,205,0,4,2008
9,1,1,9,2,2,1,1,2,2,1,...,2,1939.0,1,205,0,4,0,0,1,2008


In [27]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'subsample': 0.7,
            'subsample_freq': 1,
            'learning_rate': 0.01,
            'num_leaves': 100,
            'min_data_in_leaf':24,
            'feature_fraction': 0.9,
            'lambda_l1': 1,  
            'lambda_l2': 1
            #'device':'gpu'
            }

In [28]:

folds = 5
seed = 666

#kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=seed)

In [29]:
train_X, val_X , train_y, val_y =  train_test_split(train, target, test_size= 0.2, random_state = 666)


In [30]:
lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=categoricals)
lgb_eval = lgb.Dataset(val_X, val_y, categorical_feature=categoricals)

In [31]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10000,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=100,
                verbose_eval = 100)

Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 0.217586	valid_1's rmse: 0.214318
[200]	training's rmse: 0.147446	valid_1's rmse: 0.15889
[300]	training's rmse: 0.119777	valid_1's rmse: 0.140326
[400]	training's rmse: 0.106687	valid_1's rmse: 0.132661
[500]	training's rmse: 0.0993521	valid_1's rmse: 0.129506
[600]	training's rmse: 0.0944878	valid_1's rmse: 0.12794
[700]	training's rmse: 0.0906292	valid_1's rmse: 0.126831
[800]	training's rmse: 0.0873253	valid_1's rmse: 0.12634
[900]	training's rmse: 0.0845103	valid_1's rmse: 0.126073
[1000]	training's rmse: 0.082225	valid_1's rmse: 0.125732
[1100]	training's rmse: 0.0801284	valid_1's rmse: 0.125582
[1200]	training's rmse: 0.0783271	valid_1's rmse: 0.125365
[1300]	training's rmse: 0.0765878	valid_1's rmse: 0.125232
[1400]	training's rmse: 0.0749431	valid_1's rmse: 0.125191
[1500]	training's rmse: 0.073402	valid_1's rmse: 0.125281
Early stopping, best iteration is:
[1430]	training's rmse: 0.0744681	v

In [32]:
# fi = pd.DataFrame(  {0: train_X.columns ,  1: gbm.feature_importance()})
# low_fis = fi.sort_values(1).head(20)[0].tolist()

low_fis = ['SaleCondition','LowQualFinSF', 'SaleType', 'Heating', 'GarageQual', 'BsmtFinType2', 'GarageCond',
     'RoofMatl', '3SsnPorch', 'PoolArea', 'Condition2', 'Functional', 'PoolQC', 'Utilities', 'Street', 'MiscVal', 'BldgType', 'MiscFeature', 'RoofStyle', 'Alley']
# print(categoricals)
# for low_fi in low_fis:
#     print (low_fi)
#     features.remove(low_fi)

In [33]:
ce_test = category_encoders.OrdinalEncoder(cols=origin_categoricals,handle_unknown='impute')
test[origin_categoricals] = ce.fit_transform(test[origin_categoricals])

In [34]:
test = test[features]

In [35]:
res = gbm.predict(test)
for i in  range( len(test) ):
    res[i] = np.expm1(res[i])


In [36]:
from datetime import datetime
sub = pd.DataFrame()
sub['Id'] = test['Id']
sub['SalePrice'] = res
tmpTime = datetime.now().strftime("%Y%m%d%H%M%S")
filename='submission'+tmpTime+'.csv'
sub.to_csv( filename, index=False)
